In [46]:
#I had to create a path for pymongo because Jupyter was claiming it wasn't being found in the directory
import sys
sys.path.append("/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages")

import os
import json
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

In [9]:
#This function reads the file into a json file from a folder that I call below
def get_fantasy(folder_name):
    stats = {}
    
    for filename in os.listdir(folder_name):
        with open(os.path.join(folder_name, filename), 'r') as f: 
            content = f.read() # open in readonly mode
            #added a substitute line as python didn't like seeing $ in the input file
            content = re.sub('\$','',content)
            content_json = json.loads(content)
            filename = filename.replace('.json', '')
            content_json2 = {}
            #content_json2[filename] = content_json
            stats = content_json
    return stats

In [10]:
#This function allows us to connect to the collection within MongoDB
def save_fantasy(stats, database_name, collection_name, host_name, host_port):  
    try:
        client = pymongo.MongoClient(host_name, host_port)
        print ("Connected successfully!!!")
    except e:
        print ("Could not connect to MongoDB: %s" % e )
    else:
        client.drop_database(database_name)
        #This line allows to rerun the called functions as many times as we want without obtaining any errors
        stats_db = client[database_name]
        stats_coll = stats_db[collection_name]
        stats_coll.insert_many(stats)
        print(f"Added {len(stats)} stats to collection {collection_name}")
        # close the database connection
        client.close()

In [11]:
#This function loads the mongodb file into a pandas dataframe for easy use
def load_stats(database_name, collection_name, host_name, host_port):  
    try:
        client = pymongo.MongoClient(host_name, host_port)
        print ("Connected successfully!!!")
    except e:
        print ("Could not connect to MongoDB: %s" % e )
    else:
        
        fantasy_db = client[database_name]
        # create collection named stats
        fantasy_coll = fantasy_db[collection_name]
        #This creates the dictionary of values for the database
        stats_list = {}
        for doc in fantasy_coll.find():
            ID = doc['_id']
            year = doc['Year']
            name = doc['name']
            position = doc['position']
            team = doc['team']
            adp = doc['adp']
            player_id = doc['PlayerID']
            stats_list[ID['oid']] = ([ID['oid'], year, name, position, team, adp, player_id])
        client.close()
        #This line creates the pandas database
        df = pd.DataFrame(stats_list.values(), columns=['_id', 'year', 'name', 'position', 'team', 'adp', 'PlayerID'])
        return df

In [12]:
folder_name = 'Fantasy'
database_name = 'fantasy_football'
collection_name = 'stats'
host_name = 'localhost'
host_port = 27017
stats = get_fantasy(folder_name)
save_fantasy(stats, database_name, collection_name, host_name, host_port)

Connected successfully!!!
Added 1206 stats to collection stats


In [82]:
#This first function allows us to see the team that has the best average of players with the lowest ADP or average draft position
#This helps us indicate which team has the best success in terms of fantasy players
def team_adp_average(stats_list):
    team_group = stats_list.groupby(stats_list.team).mean().sort_values('adp',ascending=True)
    print(team_group)

#After seeing the best team is Cincinatti, this function allows us to see which of their players have had the best adp over the 7 years
def team_top_players(stats_list, team):
    players = stats_list[stats_list.team == team].sort_values('adp')
    print(players)
    
    
stats_data = load_stats(database_name, collection_name, host_name, host_port)
team_adp_average(stats_data)
team_top_players(stats_data, 'CIN')

Connected successfully!!!
             year         adp
team                         
CIN   2020.058824   71.661765
KC    2020.294118   72.390196
MIN   2020.125000   73.342500
LAR   2020.052632   74.152632
GB    2019.926829   74.317073
DAL   2020.073171   74.356098
PIT   2020.189189   75.902703
ATL   2019.710526   77.207895
PHI   2019.933333   77.268889
LAC   2020.075000   77.807500
CLE   2020.176471   77.894118
BAL   2020.218750   78.662500
SEA   2019.717949   81.041026
TEN   2019.562500   81.134375
SF    2020.255814   83.302326
NO    2019.833333   83.740476
CAR   2019.689655   84.137931
TB    2019.956522   84.852174
ARI   2020.157895   86.328947
DEN   2020.235294   86.529412
HOU   2020.031250   88.459375
LV    2020.235294   88.485294
BUF   2020.567568   89.300000
NYG   2020.030303   92.521212
JAX   2020.655172   92.555172
WAS   2019.878788   92.796970
CHI   2020.171429   92.877143
DET   2019.594595   94.635135
NE    2019.549020   97.233333
IND   2019.694444  100.530556
MIA   2020.344